## OCRパターン
基本のOCR. 
プログラムの修正は行っていないが，
oemを明示的に1と設定し，
またローカルのtesseract側で言語モデルをbestに置き換えている

In [141]:
from pytesseract import pytesseract
from PIL import Image
import glob
import re

with open("tesseract_url.txt","r") as f:
    data = f.read()
    #tesseractのパス
    pytesseract.tesseract_cmd = r"{}".format(data)

    def ocr_pattern1(i, digits, hazeer, oem, psm):

        img = Image.open(i)

        # 番号の部分を切り抜き
        img_box1 = img.crop((3230,311,4141,421))
        img_box2 = img.crop((1940,0,2392,95))

        custom_oem_psm_config = r'--oem 3 --psm 8'

        ocr_result1 = re.sub(r"\D","",pytesseract.image_to_string(img_box1, lang="eng", config=custom_oem_psm_config))    
        ocr_result2 = re.sub(r"\D","",pytesseract.image_to_string(img_box2, lang="eng", config=custom_oem_psm_config))

        return ocr_result1, ocr_result2

## 正解ラベル付きデータセットを作る

In [125]:
#得られた2つのデータのうちどちらを仮正解データにするかを設定する
def temp_label(sample1, sample2):
    if sample1 == sample2:
        return sample1
    elif sample1 != sample2:
        if len(sample2) == 10:
            return sample2
        elif len(sample1) == 10:
            return sample1
        else:
            return sample1
    else:
        return sample1


In [ ]:
from pytesseract import pytesseract
from PIL import Image
import glob
import re
import csv

with open("target_dir.txt","r") as f:
    data = f.read()
    target_dir = r"{}".format(data)
    target_file =  target_dir + "\\*.jpg"
    file_list = glob.glob(target_file) # jpgファイル一覧を取得する

    with open(r"label_data.csv","w") as w:
        writer = csv.writer(w)

        for i in file_list:
            ocr_return = ocr_pattern1(i,"a","a","a","a")
            label = temp_label(ocr_return[0],ocr_return[1])
            writer.writerow([label,i])
            print([label,i])

## 画像の傾き修正

画像の検出精度はその傾きに大きく影響されるため
OpenCVを用いて傾き修正を行う．

修正は検出された傾きが1度未満である場合のみ行う．
逆にそれ以外だと適切な修正が行えない．

In [133]:
import os
import numpy as np
import cv2

def fix_angle(input,output):

    # 画像の読み込み
    filepath = os.path.join(os.getcwd(), r"{}".format(input))
    image = cv2.imread(filepath)
    crop_img = image[150:1600, 800:4700]

    # 画像をグレースケール化して、色反転を行う（文字を白色、背景色を黒色にする）
    gray_image = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.bitwise_not(gray_image)

    # 画像の二値化
    thresh_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # 全てのピクセルで0より大きい値の座標を取得し、「cv2.minAreaRect」によって回転角を取得
    coords = np.column_stack(np.where(thresh_image > 0))
    angle = cv2.minAreaRect(coords)[-1]

    #「cv2.minAreaRect」は、[-90, 0)の領域で値を返すため、回転角を調整する
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    # 画像の歪みを補正
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0) # 回転変換行列の算出
    rotated_image = cv2.warpAffine(image, matrix, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE) # 画像のアフィン変換

    if angle > -1:
        print(angle, output)
        cv2.imwrite(r"{}".format(output),rotated_image)
    else:
        cv2.imwrite(r"{}".format(output),image)
        print(angle, output, "fail")



In [ ]:
#TestDataの中身を一括で傾き修正にかける
import re

with open("target_dir.txt","r") as f:
    data = f.read()
    target_dir = r"{}".format(data)
    target_file =  target_dir + "\\*.jpg"
    
    input_list = glob.glob(target_file) # jpgファイル一覧を取得する
    output_list = []

    for i in input_list:
        output = i.split("\\")[-1]
        output_list.append("TestData_AC\\"+output)
    
    for i in range(len(input_list)):
        fix_angle(input_list[i],output_list[i])

## 精度の検証
OCRの適切な抽出精度を算出する．
今回はデータ数が少ないので、画像一つ一つにラベル付けを行った（label_data）
このラベルに適切な値が出力されるかで評価を行う

### 評価方法
2箇所の画像領域におけるOCRの出力を用いる．
正解とする場合の判定方法は以下の通り

* img_box1と2の出力が同様ならばそれと正解ラベルを比較し判定
* 2つの出力が異なる場合10桁の出力が得られたものを用いて比較し判定
* どちらも10桁の場合は，img_box2を利用する（特に2を選んだ理由はない）

In [139]:
import csv

def check_accuracy(url):
    with open("label_data.csv","r") as f:
        reader = csv.reader(f)

        #正解リストを作成する
        answer_list = []

        for i in reader:
            answer_list.append(i)
        
        with open(url,"r") as f:
            data = f.read()
            target_dir = r"{}".format(data)
            target_file =  target_dir + "\\*.jpg"
            file_list = glob.glob(target_file) # jpgファイル一覧を取得する

            count = 0 #データの正解数を算出する
            for i in range(len(file_list)):
                ocr_return = ocr_pattern1(file_list[i],"a","a","a","a")
                label = temp_label(ocr_return[0],ocr_return[1])

                if answer_list[i][0] == label:
                    print(label)
                    count += 1
                else:
                    print("a")
            
            return count 

In [142]:
check_accuracy("target_dir.txt")

a
4221081062
4221081062
4221081062
a
4221081090
a
a
a
4221081093
4221081093
4221081093
4221081101
4221081101
4221081101
4221081101
a
4221081180
4221081180
4221081180
4221081205
4221081205
4221081205
4221081205
4221081247
a
a
4221081247
4221081251
4221081251
4221081251
4221081251
7621087210
7621087210
7621087210
7621087210
7621087211
7621087211
7621087211
7621087211
7621087212
7621087212
7621087212
7621087212
7621087213
7621087213
7621087213
a
7621087214
7621087214
7621087214
7621087214
7621087215
7621087215
7621087215
7621087215
7621087216
a
7621087216
7621087216
a
7621087217
a
7621087217
7621087218
7621087218
a
7621087218
a
7621087219
7621087219
7621087219
a
7621087220
a
7621087220
7621087221
7621087221
7621087221
7621087221
7621087222
7621087222
7621087222
7621087222
7621087223
a
7621087223
7621087223
a
a
7621087224
7621087224
7621087225
7621087225
7621087225
7621087225
a
a
7621087226
7621087226
7621087227
a
a
7621087227
7621087228
7621087228
a
7621087228
7621087229
7621087229
762108

135